In [ ]:
## VAE  (You can also choose other models such as VQVAE and MHVQVAE.)
import torch
from model.VAEs import IVAE
from scripts.train import train_IVAE
from probfun.fitness import ackley as fitness_function
from scripts.eval import evaluate, evaluate_muti_peak, set_seed

set_seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

input_dim = 2
latent_dim = 8
batch_size = 1024
epochs = 2000
learning_rate = 0.001
temperature = 10.0  


model = IVAE(input_dim, latent_dim, use_tanh=True, tanh_alpha=1/temperature).to(device)

model , loss_trace = train_IVAE(model=model, device="cpu", input_dim=input_dim, batch_size=batch_size, fitness_function=fitness_function,
                           epochs=epochs,learning_rate =learning_rate, mapping_type="sigmoid", T=temperature,
                            objective = "maximize"
                                  )


with torch.no_grad():
     z = torch.randn(1000, input_dim).to(device)
     x_0, *rest= model(z)
     
result = evaluate(model, input_dim=input_dim,obj=fitness_function, obj_name="ackley",maximize=True)

print(result)

In [ ]:
## ConsVAE

import torch
from model.VAEs import ConstrainedVAE
from scripts.train_constrained import train
from scripts.eval import set_seed, evaluate_constrained



"""
 objective function: min (x1 + 1)^2 + (x2 - 2)^2 + (x3 + 3)^2 
 s.t.
      x1^2 + x2^2 - x3 <= 0
      x1 + x2 + x3 = 1
      -2 <= xi <= 2
"""

# Define the fitness function
def fitness_function(x):
    """
    min: f(x) = (x1 + 1)^2 + (x2 - 2)^2 + (x3 + 3)^2
    x(tensor): shape: [batch_size, 3] 
    """
    x1 = x[:, 0]
    x2 = x[:, 1]
    x3 = x[:, 2]
    return (x1 + 1)**2 + (x2 - 2)**2 + (x3 + 3)**2


# Define the constraints
def constraints(x):
    """
    constraint function: 
    g(x) <= 0 and h(x) = 0
    """
    x1 = x[:, 0]
    x2 = x[:, 1]
    x3 = x[:, 2]
    
    # inequality constraints: x1^2 + x2^2 - x3 <= 0
    ineq = x1**2 + x2**2 - x3
    
    # equality constraint: x1 + x2 + x3 = 1
    eq = x1 + x2 + x3 - 1
    
    # boundary constraint: -2 <= xi <= 2 
    boundary_viol = torch.sum(F.relu(torch.abs(x) - 2), dim=1)
    
    return {
        'ineq': [ineq],
        'eq': [eq],
        'boundary': [boundary_viol]
    }

# hyper-parameter
input_dim = 3  
latent_dim = 5
batch_size = 512
epochs = 2000
initial_lr = 0.001
beta_start = 0.1
beta_end = 0.5
radius = 5.0

set_seed(42)

device = "cuda" if torch.cuda.is_available() else "cpu"

# Initialize the model 
model = ConstrainedVAE(input_dim, latent_dim)

model, loss = train(model=model,
                    device = "cpu",
                    input_dim=input_dim,
                    batch_size = batch_size,
                    fitness_function=fitness_function,
                    constraint_fn = constraints)


# test result
with torch.no_grad(): 
    z = torch.randn(10000, latent_dim)
    x_0 = model.decoder(z)
    x_0 = torch.clamp(x_0, min=-2.0, max=2.0)
    metrics = evaluate_constrained(x_0, fitness_function, constraints)

In [ ]:
## VAE-Encoder(MOE)

import torch
import torch.nn.functional as F

from probfun.mutifitness import zdt1
from model.VAEs import MOE
from scripts.eval import set_seed, evaluate_multi_objective
from scripts.train_muti_object import train


input_dim = 30
latent_dim = 60
batch_size = 256
epochs = 2000
lr = 1e-4
log_interval = 200
device = 'cuda' if torch.cuda.is_available() else 'cpu'

set_seed(42)

model = MOE(latent_dim, input_dim, activation='tanh').to(device)

model, loss = train(
    model=model,
    device=device,
    input_dim=input_dim,
    latent_dim=latent_dim,
    batch_size=batch_size,
    epochs=epochs,
    fitness_function=zdt1,
    log_interval=log_interval,
    div_weight=0.005,
    learning_rate=lr
)



model.eval()
with torch.no_grad():
    z = torch.randn(1000, latent_dim, device=device)
    x_0= model(z)
    x_0_scaled = (x_0 + 1) / 2
    f1_pre, f2_pre = zdt1(x_0_scaled)


f1_true = torch.linspace(0, 1, 100)
f2_true = 1 - torch.sqrt(f1_true)

true_pareto = torch.stack([f1_true, f2_true], dim=1)
approx_pareto = torch.stack([f1_pre, f2_pre], dim=1)

result = evaluate_multi_objective(true_pf=true_pareto, approximate_pf=approx_pareto)

print(result)